In [ ]:
import requests
import json
import pandas as pd
next_page = True
nodes = []
first=5000
offset=0
while(next_page):
    print(f"Fetching AS Rank data: first={first}, offset={offset}")
    response = requests.get(f"https://api.asrank.caida.org/v2/restful/asns/?first={first}&offset={offset}")

    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}")

    as_rank_dump = response.json()
    nodes.extend(as_rank_dump['data']['asns']['edges'])
    if not as_rank_dump['data']['asns']['pageInfo']['hasNextPage']:
        next_page = False
    offset += first

edges = [e['node'] for e in nodes]
as_rank_df = pd.DataFrame(edges)
# 1️⃣ asnDegree (dict) in eigene Spalten auflösen
asnDegree_df = as_rank_df["asnDegree"].apply(pd.Series)
asnDegree_df.columns = [f"asnDegree_{c}" for c in asnDegree_df.columns]

# 2️⃣ wieder an den Haupt-DataFrame anhängen
as_rank_df = pd.concat([as_rank_df.drop(columns=["asnDegree"]), asnDegree_df], axis=1)

# 1️⃣ asnDegree (dict) in eigene Spalten auflösen
asnCone_df = as_rank_df["cone"].apply(pd.Series)
asnCone_df.columns = [f"cone_{c}" for c in asnCone_df.columns]

# 2️⃣ wieder an den Haupt-DataFrame anhängen
as_rank_df = pd.concat([as_rank_df.drop(columns=["cone"]), asnCone_df], axis=1)

# show a quick preview
as_rank_df.head()

In [ ]:
as_rank_df.to_csv("as_rank_df.csv", index=False)